In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk

In [40]:
csv= pd.read_csv("spam.csv",encoding="ISO-8859-1")
df =pd.DataFrame(csv)
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [7]:
# check class distribution
classes = df[df.columns[0]]
print(classes.value_counts())

v1
ham     4825
spam     747
Name: count, dtype: int64


"""PREPROCESS THE DATA """


In [8]:
# convert class labels to binary values, 0 = ham  1 = spam

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

# quick check
print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: v1, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [9]:
# store SMS message data
text_messages = df[df.columns[1]]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: v2, dtype: object


In [10]:
# expressions can be found at https://regexlib.com/
# use regular expressions to replace email addresses, urls, phone numbers, etc.

# replace email addresses with 'emailaddr
processed = text_messages.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddr')

# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
# replace money symbols with 'moneysymb'
processed = text_messages.str.replace(r'£|\$', 'moneysymb')

# replace 10 digit phone numbers with 'phonenum'
processed = text_messages.str.replace(r'^[2-9]\d{2}-\d{3}-\d{4}$', 'phonenum')

# replace normal numbers with 'num'
processed = text_messages.str.replace(r'\d+(\.\d+)?', 'num')

In [11]:
# remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [12]:
# remove word stems using a Porter stemmer
# stemming is the process of reducing a word to its word stem such as removing -ing
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))

processed.head()

0    go until jurong point, crazy.. avail onli in b...
1                          ok lar... joke wif u oni...
2    free entri in 2 a wkli comp to win fa cup fina...
3    u dun say so earli hor... u c alreadi then say...
4    nah i don't think he goe to usf, he live aroun...
Name: v2, dtype: object

""" Tokenization"""


In [25]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import nltk
nltk.download('all')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package 

True

In [26]:
from nltk.tokenize import word_tokenize

# create bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

# FreqDist class is used to encode “frequency distributions”, which count the number of times that each outcome of an experiment occurs

all_words = nltk.FreqDist(all_words)

In [27]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(10)))

Number of words: 8973
Most common words: [('.', 4703), ('i', 2903), ('to', 2241), ('you', 2226), (',', 1872), ('?', 1541), ('a', 1425), ('!', 1381), ('the', 1320), ('...', 1131)]


In [28]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [29]:
# find_features function will determine which of the 1500 word features are contained in the email/message
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# example
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

go
until
jurong
point
,
crazy
..
avail
onli
in
bugi
n
great
world
la
e
buffet
...
cine
there
got
amor
wat


The above words are key words that were saved as apart of the features (aka most common words) list that were found in the very first message.

In [30]:
# do it for all the messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

In [31]:
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [32]:
print('Training:',len(training))
print('Testing:',len(testing))

Training: 4179
Testing: 1393


Alogorithms
Scikit-Learn Classifiers with NLTK

In [33]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 98.20531227566404


In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 92.39052404881551
Decision Tree Accuracy: 95.76453697056712
Random Forest Accuracy: 97.70279971284997
Logistic Regression Accuracy: 98.34888729361091


In [37]:
# ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 98.34888729361091


In [38]:
# class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [39]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1193
           1       1.00      0.79      0.88       200

    accuracy                           0.97      1393
   macro avg       0.98      0.90      0.93      1393
weighted avg       0.97      0.97      0.97      1393



predicted     
                  ham spam
actual ham       1193    0
       spam        42  158